In [1]:
import tensorflow as tf
from tensorflow import keras

In [4]:
!pip install -q -U keras-tuner

In [5]:
import keras_tuner as kt

In [6]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

4423680/4422102 [==============================] - 9s 2us/step


In [7]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

In [8]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

In [9]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

In [10]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [11]:
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 00m 33s]
val_accuracy: 0.8580833077430725

Best val_accuracy So Far: 0.8870833516120911
Total elapsed time: 00h 11m 03s
INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 192 and the optimal learning rate for the optimizer
is 0.001.



In [12]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=20, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/20
1500/1500 [==============================] - 4s 3ms/step - loss: 0.5066 - accuracy: 0.8228 - val_loss: 0.4069 - val_accuracy: 0.8550
Epoch 2/20
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3796 - accuracy: 0.8622 - val_loss: 0.3746 - val_accuracy: 0.8624
Epoch 3/20
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3408 - accuracy: 0.8755 - val_loss: 0.3445 - val_accuracy: 0.8732
Epoch 4/20
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3147 - accuracy: 0.8847 - val_loss: 0.3303 - val_accuracy: 0.8822
Epoch 5/20
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2950 - accuracy: 0.8903 - val_loss: 0.3180 - val_accuracy: 0.8849
Epoch 6/20
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2797 - accuracy: 0.8963 - val_loss: 0.3203 - val_accuracy: 0.8847
Epoch 7/20
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2663 - accuracy: 0.9002 - val_loss: 0.3139 - val_accuracy:

In [13]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/18
1500/1500 [==============================] - 4s 3ms/step - loss: 0.5107 - accuracy: 0.8221 - val_loss: 0.4391 - val_accuracy: 0.8418
Epoch 2/18
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3814 - accuracy: 0.8608 - val_loss: 0.3706 - val_accuracy: 0.8683
Epoch 3/18
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3393 - accuracy: 0.8758 - val_loss: 0.3592 - val_accuracy: 0.8695
Epoch 4/18
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3136 - accuracy: 0.8847 - val_loss: 0.3385 - val_accuracy: 0.8797
Epoch 5/18
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2972 - accuracy: 0.8900 - val_loss: 0.3338 - val_accuracy: 0.8796
Epoch 6/18
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2792 - accuracy: 0.8966 - val_loss: 0.3457 - val_accuracy: 0.8750
Epoch 7/18
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2659 - accuracy: 0.9005 - val_loss: 0.3219 - val_accuracy:

In [14]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

313/313 [==============================] - 1s 3ms/step - loss: 0.3521 - accuracy: 0.8909
[test loss, test accuracy]: [0.35213911533355713, 0.8909000158309937]


# OBSERVATION

In this work i have understood that Accuracy will depend on number of epochs as we increase the epochs the time it will take is much more.From the above i have runned 20 epochs out of which 18th one gives more accuracy.As we increase the epochs the model may try to overfit

# CHALLENGES IN HYPERPARAMETERS TURING

All the hyper parameters will depend on dataset.if the dataset is small the learning has to happen slowly. so we cant give learning rate as 0.1 or 1.0 . if the dataset is have this happens vice versa. The momentum also as we know it should be 0.9 . And if we give random state as 0 , it will work properly for most of the datasets and some will doesn't work . So all these we can't predict what to choose at what time . if we give small values the model will suffer like anything .The complexity increases and it will take lot of time . so this can be hectic work to choose the parameters. By adding more layers and more units within a layer, a deep network can represent functions of increasing complexity ,Neural networks are complex linear functions with many parameters. 6. A perceptron adds up all the weighted inputs it receives, and if it exceeds a certain value, it outputs a 1, otherwise it just outputs a 0.Neural complexity deals with lower bounds for neural resources (numbers of neurons) needed by a network to perform a given task within a given tolerance.